# This is to test the code of the AL loop

Eventualy will be in a script but there is a lot to put together and I want to be able to run this line by line

In [48]:
from glob import glob
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score
from mlflow.models.signature import infer_signature
from datetime import datetime
import os
from finkvra.utils.features import make_features as fvra_make_features
from finkvra.utils.labels import cli_label_one_object as fvra_cli_label_one_object
import json
from mlflow.tracking import MlflowClient
import mlflow
import logging

In [49]:
EXPERIMENT = "gal_model_AL2"
SAMPLING_STRATEGY = "uncertainty"
mlflow.set_tracking_uri("http://127.0.0.1:6969")
mlflow.set_experiment(EXPERIMENT)

If get ``[WARNING] .... Errno 111 Connection refused`` need to make sure we start the server

```bash
mlflow server --host 127.0.0.1 --port 6969
```

In [50]:
# --------------
# 0. Set up 
# --------------

client = MlflowClient()
experiment = client.get_experiment_by_name(EXPERIMENT)

In [51]:

experiment_id = experiment.experiment_id

runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="attributes.status = 'FINISHED'",
    order_by=["start_time DESC"],
    max_results=1,
)

if not runs:
    raise RuntimeError("No successful (FINISHED) runs found.")

last_run = runs[0]
prev_run_id = last_run.info.run_id
print("Last successful run ID:", prev_run_id)


Last successful run ID: 800ea0b1efc84d93b737a6b12c7577e6


In [52]:
# -------------------
# 1. Load data
# -------------------
files = sorted(glob('/home/stevance/Data/FinkZTFStream/*.parquet'))
dfs = [pd.read_parquet(f) for f in files]
data = pd.concat(dfs, ignore_index=True)


In [53]:
data.head(2)

,candid,objectId,ra,dec,drb,mjd,mag,maglim,fid,isdiffpos,lc_features_g,lc_features_r,sherl_class,sep_arcsec
0,3069386415315015051,ZTF25aastlyj,288.662154,65.088530,0.961113,"[60823.38641199982, 60795.374525499996, 60795....","[19.527942657470703, nan, nan, nan, nan, nan, ...","[20.746294021606445, 20.559799194335938, 20.42...","[1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, ...","[t, None, None, None, None, None, None, None, ...","{'amplitude': 0.0, 'anderson_darling_normal': ...","{'amplitude': None, 'anderson_darling_normal':...",ORPHAN,nan
1,3069387894915010012,ZTF25aastmkd,341.846733,72.183242,0.994998,"[60823.38789349981, 60795.426064800005, 60795....","[19.923898696899414, nan, nan, nan, nan, nan, ...","[20.43282127380371, 20.07029914855957, 20.3325...","[1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, ...","[f, None, None, None, None, None, None, None, ...","{'amplitude': 0.0, 'anderson_darling_normal': ...","{'amplitude': None, 'anderson_darling_normal':...",ORPHAN,nan


In [54]:
# -------------------
# 2. Make our features
# -------------------
X, meta = fvra_make_features(data)
# remove samples that have no positive detections
valid_candid = list(X[X.ndets > 0].index.values)
X = X.loc[valid_candid]
meta = meta.loc[valid_candid]


In [55]:
X.shape[0], meta.shape[0]

(92, 92)

In [56]:
X.head()

,ra,dec,drb,ndets,nnondets,dets_median,dets_std,sep_arcsec,amplitude,linear_fit_reduced_chi2,linear_fit_slope,linear_fit_slope_sigma,median,median_absolute_deviation,amplituder_,linear_fit_reduced_chi2r_,linear_fit_sloper_,linear_fit_slope_sigmar_,medianr_,median_absolute_deviationr_
candid,,,,,,,,,,,,,,,,,,,,
3069386415315015051,288.662154,65.088530,0.961113,1.0,17.0,19.527943,0.000000,nan,0.000000,NaN,NaN,NaN,19.527943,0.000000,0.000000,NaN,NaN,NaN,19.527943,0.000000
3069382582215015014,293.757494,24.732624,0.999734,1.0,17.0,19.765827,0.000000,1.2398935354229732,0.000000,NaN,NaN,NaN,19.765827,0.000000,0.000000,NaN,NaN,NaN,19.765827,0.000000
3069386410515015007,286.972916,60.047391,0.999990,11.0,6.0,20.475401,0.075382,0.42265986534096434,0.070496,0.069703,0.003666,0.004152,19.842197,0.018932,0.070496,0.069703,0.003666,0.004152,19.842197,0.018932
3069388402015015013,334.137737,46.956847,0.995750,1.0,16.0,19.520535,0.000000,9.0563054064397,0.000000,NaN,NaN,NaN,19.520535,0.000000,0.000000,NaN,NaN,NaN,19.520535,0.000000
3069388403815015014,332.785467,48.254230,0.999997,5.0,11.0,19.384800,0.150037,nan,0.152000,0.426425,0.069098,0.047245,19.384800,0.036600,0.152000,0.426425,0.069098,0.047245,19.384800,0.036600


In [57]:
meta.head()

,objectId
candid,
3069386415315015051,ZTF25aastlyj
3069382582215015014,ZTF25aastlfp
3069386410515015007,ZTF19aarxyde
3069388402015015013,ZTF25aastlnm
3069388403815015014,ZTF21abiumls


In [58]:
# -------------------
# 3. Load previous training set and exclude
# -------------------
# Artifact: we log and load training IDs as a CSV
previous_ids_path = "./training_ids.csv"
previous_ids_df = pd.read_csv(previous_ids_path)
CURRENT_ROUND= previous_ids_df.iloc[-1]['round'] + 1

train_ids = previous_ids_df["candid"].tolist()
X_pool = X.drop(index=train_ids, errors='ignore')
meta_pool = meta.drop(index=train_ids, errors='ignore')

In [59]:
previous_ids_df

,candid,round
0,3069386410515015007,0
1,3069382582215015014,0
2,3069388402015015013,0
3,3069382583515015017,0
4,3069386414215015010,0
5,3069387892715015046,0
6,3069383052515015018,0
7,3069388403815015014,0
8,3070411123815015010,0
9,3070413103915015007,0


In [60]:
X_pool.shape

(52, 20)

In [61]:
mlflow_uri = mlflow.get_tracking_uri()

In [62]:
prev_run_id

'800ea0b1efc84d93b737a6b12c7577e6'

In [63]:
# -------------------
# 4. Load previous model and predict
# -------------------
model_subpath = "gal_model"
model_uri = f"runs:/{prev_run_id}/{model_subpath}"
clf = mlflow.sklearn.load_model(model_uri)
y_pred = clf.predict_proba(X_pool)[:, 1]  # assuming binary classifications


This will hang forever if the files don't exist

In [64]:
y_pred_pool = pd.DataFrame( np.vstack((X_pool.index.values.astype(str), y_pred)).T, columns= ['candid', 'pred']).set_index('candid')

In [65]:
y_pred_pool.pred

candid
3069386415315015051     0.6558614194626307
3069386413215015028       0.66324666173808
3069383051615015019     0.6854525015143842
3069383054815015012     0.6783180537153045
3069386413215015015       0.66324666173808
3070410643915015008       0.56949704392139
3070411122915015028       0.56949704392139
3070411123815015009     0.5445444682142248
3070412073315015009     0.5445444682142248
3070411594015015000     0.5363757138012302
3070412073515015022     0.6558614194626307
3070411592515015014     0.5445444682142248
3070412075015015028     0.5363757138012302
3070411593315015008     0.4477056803206498
3070411595315015032     0.5445444682142248
3070412552815015017     0.4093986026750948
3070411125015015041     0.4439099486465521
3070412075615015039     0.5363757138012302
3070412075615015029    0.23913041208230745
3070417502615015020     0.5963663630020621
3070416043015015020    0.42459169525241147
3070415083215015020     0.6558614194626307
3070416523515015032     0.6854525015143842
3070

In [66]:
# -------------------
# 6. Active Learning sampling
# -------------------
y_pred_pool['uncertainty_score'] = np.abs(y_pred_pool["pred"].astype(float) - 0.5) 

In [67]:
y_pred_pool = y_pred_pool.sort_values('uncertainty_score', ascending=True)
y_pred_pool

,pred,uncertainty_score
candid,,
3070417984115015023,0.49413252172253985,0.005867
3070412075615015039,0.5363757138012302,0.036376
3070412075015015028,0.5363757138012302,0.036376
3070411594015015000,0.5363757138012302,0.036376
3070411592515015014,0.5445444682142248,0.044544
3070411595315015032,0.5445444682142248,0.044544
3070411123815015009,0.5445444682142248,0.044544
3070412073315015009,0.5445444682142248,0.044544
3070420385215015013,0.4504532049245878,0.049547


In [68]:
# -------------------
# 7. Labels 
# -------------------

labels = pd.read_csv('~/Data/FinkZTFStream/labeled.csv', index_col=0)

In [69]:
labels

,objectId,label,timestamp
candid,,,
3069386410515015007,ZTF19aarxyde,varstar,2025-06-02T08:37:22.993787
3069386415315015051,ZTF25aastlyj,real,2025-06-02T08:38:09.167712
3069382582215015014,ZTF25aastlfp,varstar,2025-06-02T08:38:35.964541
3069388402015015013,ZTF25aastlnm,varstar,2025-06-02T08:38:48.162550
3069382583515015017,ZTF25aastmlg,varstar,2025-06-02T09:44:44.227317
...,...,...,...
3070420385215015013,ZTF25aasfmrt,real,2025-06-04T06:35:11.160241
3070412070215015010,ZTF23aagfkmu,varstar,2025-06-04T06:35:11.160241
3070412070715015000,ZTF25aaortzp,extragal,2025-06-04T06:35:11.160241


In [70]:
label2galclass = {'real': np.nan, 
                  'extragal': 0, 
                  'gal': 1, 
                  'agn': 0,
                  'bogus': np.nan, 
                  'varstar': 1,
                  None: np.nan
                 }

In [71]:
new_labels = []
new_label_candid = []
new_sample_candid = []

N_to_sample = 10
N_i = 0

for _candid in y_pred_pool.index.astype(np.int64):
    try: 
        classification = label2galclass[labels.loc[_candid].label]

        if not np.isnan(classification):
            new_sample_candid.append(_candid)
            N_i += 1
    except KeyError: 
        # if _candid not in labels then labels.loc[_candid] will throw a KeyError 
        # and we can activate the logic below
        _objectId = meta.loc[_candid].objectId
        
        # this is where we need the labeling 
        label = fvra_cli_label_one_object(_objectId)
        if label is None: 
            continue
            
        new_labels.append(label)
        new_label_candid.append(_candid)
        classification = label2galclass[label]
        if not np.isnan(classification):
            new_sample_candid.append(_candid)
            N_i += 1
        
    if N_i == N_to_sample:
        break

if N_i < N_to_sample:
    print('not enough ')


 Object: ZTF25aatbcwo
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF25aatbcwo...
Label [r/x/g/a/b/v] (s=skip, q=quit): s
Skipped.

 Object: ZTF25aappflz
Opening https://lasair-ztf.lsst.ac.uk/objects/ZTF25aappflz...
Label [r/x/g/a/b/v] (s=skip, q=quit): g
Labeled as 'gal'


In [72]:
N_i

10

In [73]:
y_pred_pool.index.astype(np.int64)

Index([3070417984115015023, 3070412075615015039, 3070412075015015028,
       3070411594015015000, 3070411592515015014, 3070411595315015032,
       3070411123815015009, 3070412073315015009, 3070420385215015013,
       3070411593315015008, 3070412553215015030, 3070411125015015041,
       3070411122915015028, 3070410643915015008, 3070412070215015010,
       3070416043015015020, 3070412552815015017, 3070412070715015000,
       3070417502615015020, 3070410640115015056, 3070418963915015013,
       3070416526315015021, 3070412074815015002, 3069386415315015051,
       3070415083215015020, 3070412073515015022, 3069386413215015028,
       3069386413215015015, 3070419905315015025, 3069383054815015012,
       3070420380915015011, 3070416523515015032, 3070417981715015037,
       3069383051615015019, 3070419905315015021, 3070413103215015021,
       3070420384315015019, 3070419442415015034, 3070419441815015028,
       3070412075615015029, 3070412073215015016, 3070412550115015030,
       3070412550615

In [74]:
meta

,objectId
candid,
3069386415315015051,ZTF25aastlyj
3069382582215015014,ZTF25aastlfp
3069386410515015007,ZTF19aarxyde
3069388402015015013,ZTF25aastlnm
3069388403815015014,ZTF21abiumls
...,...
3070416526315015021,ZTF25aappflz
3070419905315015021,ZTF25aatbgvl
3070420380915015011,ZTF23aaormpw


In [75]:
timestamp = datetime.utcnow().isoformat()
new_labels_df = pd.DataFrame.from_dict({
                                         'objectId': meta.loc[np.array(new_label_candid).astype(np.int64)].objectId,
                                          'label':  new_labels,
                                        'timestamp': timestamp
                                       })

In [76]:
new_labels_df

,objectId,label,timestamp
candid,,,
3070416526315015021,ZTF25aappflz,gal,2025-06-04T06:40:09.726084


In [77]:
updated_labels = pd.concat((labels, new_labels_df))
updated_labels

,objectId,label,timestamp
candid,,,
3069386410515015007,ZTF19aarxyde,varstar,2025-06-02T08:37:22.993787
3069386415315015051,ZTF25aastlyj,real,2025-06-02T08:38:09.167712
3069382582215015014,ZTF25aastlfp,varstar,2025-06-02T08:38:35.964541
3069388402015015013,ZTF25aastlnm,varstar,2025-06-02T08:38:48.162550
3069382583515015017,ZTF25aastmlg,varstar,2025-06-02T09:44:44.227317
...,...,...,...
3070412070215015010,ZTF23aagfkmu,varstar,2025-06-04T06:35:11.160241
3070412070715015000,ZTF25aaortzp,extragal,2025-06-04T06:35:11.160241
3070410640115015056,ZTF20aauxojg,agn,2025-06-04T06:35:11.160241


In [78]:
updated_labels.to_csv('~/Data/FinkZTFStream/labeled.csv')

In [79]:
new_ids_df = pd.DataFrame({'candid': np.array(new_sample_candid).astype(np.int64),
                           'round': CURRENT_ROUND,
                          })
new_ids_df

,candid,round
0,3070417984115015023,1
1,3070412553215015030,1
2,3070411125015015041,1
3,3070412070215015010,1
4,3070416043015015020,1
5,3070412552815015017,1
6,3070412070715015000,1
7,3070410640115015056,1
8,3070418963915015013,1
9,3070416526315015021,1


In [80]:
previous_ids_df

,candid,round
0,3069386410515015007,0
1,3069382582215015014,0
2,3069388402015015013,0
3,3069382583515015017,0
4,3069386414215015010,0
5,3069387892715015046,0
6,3069383052515015018,0
7,3069388403815015014,0
8,3070411123815015010,0
9,3070413103915015007,0


In [81]:
train_ids_df = pd.concat([previous_ids_df, new_ids_df]).reset_index(drop=True)
train_ids_df.tail(15)

,candid,round
35,3070409643215015025,0
36,3070414063615015016,0
37,3070410151215015002,0
38,3070412555215015027,0
39,3070413104315015004,0
40,3070417984115015023,1
41,3070412553215015030,1
42,3070411125015015041,1
43,3070412070215015010,1
44,3070416043015015020,1


In [89]:
train_ids_df.to_csv('./training_ids.csv', index=False)


train_ids_df.to_csv

In [82]:
y_train = updated_labels.loc[train_ids_df.candid].label.map(label2galclass)

In [83]:
X_train = X.loc[train_ids_df.candid]

In [84]:
X_train

,ra,dec,drb,ndets,nnondets,dets_median,dets_std,sep_arcsec,amplitude,linear_fit_reduced_chi2,linear_fit_slope,linear_fit_slope_sigma,median,median_absolute_deviation,amplituder_,linear_fit_reduced_chi2r_,linear_fit_sloper_,linear_fit_slope_sigmar_,medianr_,median_absolute_deviationr_
candid,,,,,,,,,,,,,,,,,,,,
3069386410515015007,286.972916,60.047391,0.999990,11.0,6.0,20.475401,0.075382,0.42265986534096434,0.070496,0.069703,0.003666,0.004152,19.842197,0.018932,0.070496,0.069703,0.003666,0.004152,19.842197,0.018932
3069382582215015014,293.757494,24.732624,0.999734,1.0,17.0,19.765827,0.000000,1.2398935354229732,0.000000,NaN,NaN,NaN,19.765827,0.000000,0.000000,NaN,NaN,NaN,19.765827,0.000000
3069388402015015013,334.137737,46.956847,0.995750,1.0,16.0,19.520535,0.000000,9.0563054064397,0.000000,NaN,NaN,NaN,19.520535,0.000000,0.000000,NaN,NaN,NaN,19.520535,0.000000
3069382583515015017,296.641470,26.630707,0.951255,1.0,16.0,19.625126,0.000000,3.1566587143809444,0.000000,NaN,NaN,NaN,19.625126,0.000000,0.000000,NaN,NaN,NaN,19.625126,0.000000
3069386414215015010,283.759268,62.913423,0.999961,1.0,16.0,20.100019,0.000000,5.758135128184068,0.000000,NaN,NaN,NaN,19.690671,0.000000,0.000000,NaN,NaN,NaN,19.690671,0.000000
3069387892715015046,332.226887,67.619276,0.999982,15.0,1.0,19.994400,0.386317,0.47233313806431115,0.699652,17.086155,0.007330,0.004989,19.697263,0.419206,0.699652,17.086155,0.007330,0.004989,19.697263,0.419206
3069383052515015018,298.880915,25.674268,0.999996,2.0,12.0,20.086121,0.231821,0.20376269949088832,0.203694,NaN,NaN,NaN,19.943809,0.203694,0.203694,NaN,NaN,NaN,19.943809,0.203694
3069388403815015014,332.785467,48.254230,0.999997,5.0,11.0,19.384800,0.150037,nan,0.152000,0.426425,0.069098,0.047245,19.384800,0.036600,0.152000,0.426425,0.069098,0.047245,19.384800,0.036600
3070411123815015010,263.896232,48.273891,0.999446,11.0,7.0,20.564301,0.176693,0.4867922349335001,0.080833,0.129013,0.001508,0.006303,20.134853,0.032079,0.080833,0.129013,0.001508,0.006303,20.134853,0.032079


In [85]:
X_train.shape, y_train.shape

((50, 20), (50,))

### Train and log with ML Flow

In [86]:
CURRENT_ROUND

1

In [87]:
MODEL_TAG = f"gal_model_round_{CURRENT_ROUND}"

with mlflow.start_run(run_name=f"round_{CURRENT_ROUND}_{SAMPLING_STRATEGY}"):

    # Log metadata
    meta_info = {
        "round": int(CURRENT_ROUND),
        "timestamp": datetime.utcnow().isoformat(),
        "n_train": int(X_train.shape[0]),
        "sampling_strategy": str(SAMPLING_STRATEGY),
        "model_tag": str(MODEL_TAG)
    }
    with open("meta.json", "w") as f:
        json.dump(meta_info, f, indent=2)
    mlflow.log_artifact("meta.json")

    # Train model
    clf_new = HistGradientBoostingClassifier(max_iter=100, 
                                             l2_regularization=10,
                                             random_state=42,
                                             learning_rate=0.1)
    clf_new.fit(X_train.values, y_train.values)

    # Evaluate on training set
    acc = accuracy_score(y_train, clf_new.predict(X_train.values))
    mlflow.log_metric("train_accuracy", acc)

    # Log model
    signature = infer_signature(X_train, clf_new.predict(X_train))
    mlflow.sklearn.log_model(
        clf_new,
        artifact_path="gal_model",
        signature=signature,
        input_example=X_train.iloc[:2]
    )

    # Save training state
    mlflow.log_artifact("training_ids.csv")

/home/stevance/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


🏃 View run round_1_uncertainty at: http://127.0.0.1:6969/#/experiments/0/runs/48c4aac3f3504f28a503ccc026e14286
🧪 View experiment at: http://127.0.0.1:6969/#/experiments/0


/home/stevance/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(
